### 자동 미분 (torch.autograd)

- PyTorch의 autograd 는 신경망 훈련을 지원하는 자동 미분 가능
- torch.autograd 동작 방법
    - 텐서에 .requires_grad 속성을 True로 설정하면, 이후의 텐서 모든 연산들을 추적함
    - 텐서.backward() 를 호출하면, 연산에 연결된 각 텐서들의 미분 값을 계산하여, 각 텐서 객체에 .grad에 저장
        - .requires_grad_()는 연결된 Tensor로부터의 계산된 자동미분 값을, 다시 현 텐서부터 시작하도록 만듦

### 신경망 동작 이해

- 모델 및 데이터 생성
- forward pass로 입력 데이터를 모델에 넣어서 예측값 계산
- 예측값과 실제값의 차이를 loss function 으로 계산
- backward pass 로 각 모델 파라미터를 loss 값 기반 미분하여 저장
- optimizer 로 모델 파라미터의 최적값을 찾기 위해, 파라미터 값 업데이트

- 텐서에 .requires_grad 속성을 True 로 설정
- .requires_grad 속성이 True 로 설정되면, 텐서의 모든 연산 추적을 위해, 내부적으로 방향성 비순환 그래프(DAG : Directed Acyclic Graph)를 동적 구성
    - 방향성 비순환 그래프(DAG)의 leaf 노드는 입력 텐서이고, root 노드는 결과 텐서가 됨

In [3]:
import torch

x = torch.rand(1, requires_grad=True)
y = torch.rand(1)
y.requires_grad = True
loss = y - x

### 🔁 텐서의 `.backward()` 동작 설명

- `tensor.backward()` 를 호출하면,  
  연산에 연결된 각 텐서들의 **미분 값(gradient)**을 자동으로 계산하여  
  각 텐서 객체의 `.grad` 속성에 저장된다.

---

### 🎯 예시

$$
\frac{\partial \text{Loss}}{\partial x} = -1, \quad \frac{\partial \text{Loss}}{\partial y} = 1
$$

> 즉, Loss를 기준으로 각 입력값에 대한 **기울기(gradient)**가 계산되어  
> `.grad` 속성에 자동으로 저장된다.

In [4]:
loss.backward()
print(x.grad, y.grad)

tensor([-1.]) tensor([1.])


In [6]:
x = torch.ones(4)
y = torch.zeros(3)
W = torch.rand(4, 3, requires_grad=True)
b = torch.rand(3, requires_grad=True)
z = torch.matmul(x,W) + b
print(W, b, z)

tensor([[0.0033, 0.4749, 0.2218],
        [0.8056, 0.3840, 0.0867],
        [0.5820, 0.8906, 0.5938],
        [0.6829, 0.3664, 0.6069]], requires_grad=True) tensor([0.5548, 0.9362, 0.5315], requires_grad=True) tensor([2.6285, 3.0521, 2.0408], grad_fn=<AddBackward0>)


In [7]:
import torch.nn.functional as F

loss = F.mse_loss(z, y)
loss.backward()
print(loss, W.grad, b.grad)


tensor(6.7964, grad_fn=<MseLossBackward0>) tensor([[1.7524, 2.0347, 1.3605],
        [1.7524, 2.0347, 1.3605],
        [1.7524, 2.0347, 1.3605],
        [1.7524, 2.0347, 1.3605]]) tensor([1.7524, 2.0347, 1.3605])


In [ ]:
threshold = 0.1
learning_rate = 0.1
iteration_num = 0

while loss > threshold :
    iteration_num += 1
    W = W - learning_rate * W.grad
    b = b - learning_rate * b.grad
    print(iteration_num, loss, z, y)
    
    # detach_() : 텐서를 기존 방향성 비순환 그래프(DAG : Directed Acyclid Graph) 로부터 끊음
    # .requires_grad(True) : 연결된 Tensor 로부터의 계산된 자동미분 값을, 다시 현 텐서부터 시작하도록 만듦
    W.detach_().requires_grad_(True)
    b.detach_().requires_grad_(True)

    z = torch.matmul(x, W) + b
    loss = F.mse_loss(z, y)
    loss.backward()

print(iteration_num + 1, loss, z, y)

1 tensor(6.7964, grad_fn=<MseLossBackward0>) tensor([2.6285, 3.0521, 2.0408], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
2 tensor(3.0206, grad_fn=<MseLossBackward0>) tensor([1.7524, 2.0347, 1.3605], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
3 tensor(1.3425, grad_fn=<MseLossBackward0>) tensor([1.1682, 1.3565, 0.9070], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
4 tensor(0.5967, grad_fn=<MseLossBackward0>) tensor([0.7788, 0.9043, 0.6047], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
5 tensor(0.2652, grad_fn=<MseLossBackward0>) tensor([0.5192, 0.6029, 0.4031], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
6 tensor(0.1179, grad_fn=<MseLossBackward0>) tensor([0.3461, 0.4019, 0.2687], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
7 tensor(0.0524, grad_fn=<MseLossBackward0>) tensor([0.2308, 0.2679, 0.1792], grad_fn=<AddBackward0>) tensor([0., 0., 0.])
